## How to use Mathematical Program, and use it to formulate optimizations for robots

One area of tools offered in pydrake is the MathematicalProgram interface.  MathematicalProgram is a class that abstracts many different useful types of optimization solvers.  This makes it so from one interface, you can access many different solvers.  The MathematicalProgram component of Drake is comparable to JuMP, in the Julia ecosystem. To get a concise overview of which solvers are supported for which different types of optimization problems, check out [this chart](http://drake.mit.edu/doxygen_cxx/group__solvers.html).  

As is the case with hot-off-the-presses code, there is not yet a plethora of documentation, but here we're going to try to provide you with enough sample code to help get you started. 

In addition to the code snippets below, these two tips are also very useful:

- Once you construct a MathematicalProgram object, i.e. `mp = MathematicalProgram()`, the tab completion in your jupyter notebook can be very helpful.  

 --> For example, let's say you want to know if a MathematicalProgram can print how many decision variables currently exist.  Tab completing on `mp.` and scrolling through, you'll find `num_vars`.  Indeed `mp.num_vars()` will do the trick.
 
 --> Want to know which solver MP is currently using under the hood for a particular problem instance?
 
```python
result = Solve(mp)
print result.is_success()
print result.GetSolution(x)
print result.get_solver_id().name()
```
- An additional resource for how to use MathematicalProgram is the tests written for it. There are a significant amount of tests for MathematicalProgram, written in C++.  See [here](https://github.com/RobotLocomotion/drake/blob/master/solvers/test/mathematical_program_test.cc) but also other tests in that folder.  Note however that not all C++ features have pydrake bindings -- for those familiar with pybind, the bindings for MathematicalProgram are generated [here](https://github.com/RobotLocomotion/drake/blob/master/bindings/pydrake/solvers/mathematicalprogram_py.cc), and are demonstrated in numerous tests [here](https://github.com/RobotLocomotion/drake/tree/master/bindings/pydrake/solvers/test). Obviously though the features  demonstrated below all have pydrake bindings.  There are sufficient features currently in pydrake to satisfiably complete the problem set.

### Okay, but how do I actually do an optimization problem?  

How do we translate something written on the board as a linear program, and write it down in code?

Here is a very simple example of an LP:

\begin{align*}
        \min_{x} \ \ \ x \\
        s.t. \ \ \  & x >= 1 \\
\end{align*}

And the corresponding Mathematical Program code is below.

In [29]:
from pydrake.all import MathematicalProgram, Solve, Variables, Polynomial
import numpy as np
import math

In [30]:
mp = MathematicalProgram()
x = mp.NewContinuousVariables(1, "x")
mp.AddLinearCost(x[0]*1.0)
mp.AddLinearConstraint(x[0] >= 1)
result = Solve(mp)
print result.is_success()
print result.GetSolution(x)

True
[ 1.]


Note that written down slightly incorrectly, you will not get the answer you were looking for.  What is wrong about the two examples below?

In [31]:
mp = MathematicalProgram()
x = mp.NewContinuousVariables(1, "x")
mp.AddLinearCost(x[0]*1.0)
result = Solve(mp)
print result.is_success()
print result.GetSolution(x)

False
[ nan]


In [32]:
mp = MathematicalProgram()
x = mp.NewContinuousVariables(1, "x")
mp.AddLinearCost(x[0]*1.0)
mp.AddLinearConstraint(x[0] <= 1)
result = Solve(mp)
print result.is_success()
print result.GetSolution(x)

False
[ -6.17673396e+14]


Here's a slightly more complicated example, this one solves a problem that may look familiar to you.

This is just one example of how, even though Linear Programs can only handle linear objectives and constraints, you can use them to sample over arbitrarily complex functions, and the samples of those functions can still be just linear constraints / objectives.

In [33]:
mp = MathematicalProgram()
alpha = mp.NewContinuousVariables(1, "alpha")
mp.AddLinearCost(alpha[0]*1.0)
for xi in np.arange(-5*np.pi, 5*np.pi+np.pi/8, np.pi/8):
    mp.AddLinearConstraint(alpha[0] - math.cos(xi)**2 + math.sin(xi) >= 0)
    
result = Solve(mp)
print result.is_success()
print result.GetSolution(alpha)

True
[ 1.23623682]


Note the MathematicalProgram is formulated in terms of "costs", and will minimize the objective function's costs when calling `Solve()`.  How can we maximize functions? Just add a negative sign:

In [34]:
mp = MathematicalProgram()
x = mp.NewContinuousVariables(1, "x")
mp.AddLinearCost(-x[0]*1.0)
mp.AddLinearConstraint(x[0] <= 4)
result = Solve(mp)
print result.is_success()
print result.GetSolution(x)

True
[ 4.]


Now how about if we want to go outside the realm of Linear Programs?  What if we want to do a Quadratic Program?  Recall that the only difference between a quadratic program and a Linear Program is that QPs now allow a quadratic cost, but still only linear objectives.

In [35]:
mp = MathematicalProgram()
x = mp.NewContinuousVariables(1, "x")
mp.AddQuadraticCost((x[0]-3)**2)
result = Solve(mp)
print result.is_success()
print result.GetSolution(x)

True
[ 3.]


Note that as above, a QP can be well formulated even without any constraints.  (LPs will have unbounded objectives without constraints.)

Note that there is no `QuadraticConstraint` in MathematicalProgram.  Why not?  (What class of problem is a QuadraticConstraint?)

But actually although there is no specific function call for it, MathematicalProgram can generally handle a quadratic constraint, and many other different types of constraints, through `AddConstraint`, where inside the argument to the function is a symbolic formula of type `==`, `>=`, or `<=`.  This opens up MathematicalProgram to solve general nonlinear optimization problems.

In [36]:
mp = MathematicalProgram()
x = mp.NewContinuousVariables(2, "x")
mp.AddConstraint((x**2).sum() == 1.)
mp.AddLinearCost(x.sum())
result = Solve(mp)
print result.is_success()
print result.GetSolution(x)

False
[ 0.  0.]


Alternatively you can even use many numpy operations, including `dot`:

In [37]:
mp = MathematicalProgram()
x = mp.NewContinuousVariables(2, "x")
mp.AddConstraint(x.dot(x) == 1.)
mp.AddLinearCost(x.sum())
result = Solve(mp)
print result.is_success()
print result.GetSolution(x)

False
[ 0.  0.]


Note that you can print out useful prints at many steps of interacting with Mathematical Program, for example:

In [38]:
mp = MathematicalProgram()
x = mp.NewContinuousVariables(2, "x")
print type(x)
print x

<type 'numpy.ndarray'>
[Variable('x(0)', Continuous) Variable('x(1)', Continuous)]


In [39]:
y = x**2
print y

[<Expression "pow(x(0), 2)"> <Expression "pow(x(1), 2)">]


# Sum-of-square optimization
The sum of square optmization is also supported in the MathematicalProgram interface.

However, it looks a little bit different. Let's start with a simple one.

In [40]:
prog = MathematicalProgram()
x = prog.NewIndeterminates(1, "x")
V = x.dot(x) + 1
print 'The type of x[0] is', type(x[0])
print 'The type of V is ', type(V)
prog.AddSosConstraint(V)
result = Solve(prog)
print result.is_success()

The type of x[0] is <class 'pydrake.symbolic.Variable'>
The type of V is  <class 'pydrake.symbolic.Expression'>
True


You can think of the `x` above as a monomial, `V` is a polynomial built with `x`. 
The `V` is a symbolic expression and `x` is a numpy array contains symbolic variable.

Now try another example

In [41]:
prog = MathematicalProgram()
x = prog.NewIndeterminates(1, "x")
V = x.dot(x - 10) + 1
prog.AddSosConstraint(V)
result = Solve(prog)
print result.is_success()

False


Clearly, $x(x - 10) + 1$ doesn't have a sum of square decomposition. Thus, the constraint is no longer feasible.

The V functions in the previous examples do not have any decision variables. Sometimes you might want to parameterize the function using a polynomial with given order.

In [59]:
prog = MathematicalProgram()
x = prog.NewIndeterminates(1, "x")
order = 4
poly = prog.NewFreePolynomial(Variables(x), order)
print poly

a(4)*1 + a(0)*x(0)^4 + a(1)*x(0)^3 + a(2)*x(0)^2 + a(3)*x(0)


The `a` is the coefficient of the polynomial, usually they are the decision variables. If we want our polynomial to be SOS, we can do the following.

In [69]:
prog = MathematicalProgram()
x = prog.NewIndeterminates(1, "x")
order = 4
poly = prog.NewFreePolynomial(Variables(x), order)
prog.AddSosConstraint(poly)
result = Solve(prog)
print result.is_success()

True


Equivalently, you can directly define a SOS polynomial. Note that there is something subtle with the type

In [70]:
prog = MathematicalProgram()
x = prog.NewIndeterminates(1, "x")
order = 4
(lambda_sos, constraint) = prog.NewSosPolynomial(Variables(x), order)
print 'The type of lambda is ', type(lambda_sos)
print 'The type can be used in prog.AddXXXXConstraint is ', type(lambda_sos.ToExpression())
prog.AddSosConstraint(-lambda_sos.ToExpression())
result = Solve(prog)
print result.is_success()

The type of lambda is  <class 'pydrake.symbolic.Polynomial'>
The type can be used in prog.AddXXXXConstraint is  <class 'pydrake.symbolic.Expression'>
True


Now we are ready to solve a more interesting SOS optimization. The example below verifies the stability of a one dimensional system. This example is from the textbook.

In [72]:
import math
from pydrake.all import (Jacobian, SolutionResult, Variables)

def dynamics(x):
    return -x + x**3

# Construct the program
prog = MathematicalProgram()
x = prog.NewIndeterminates(1, "x")
rho = prog.NewContinuousVariables(1, "rho")[0]

# Define the Lyapunov function.
V = x.dot(x)
Vdot = Jacobian([V],x).dot(dynamics(x))[0]

# Define the Lagrange multipliers.
(lambda_sos, constraint) = prog.NewSosPolynomial(Variables(x), 4)

# Add sum of square constraint
prog.AddSosConstraint((V-rho) * x.dot(x) - lambda_sos.ToExpression() * Vdot)
prog.AddLinearCost(-rho)

# Solve it
result = Solve(prog)
assert(result.is_success())
print("Verified that " + str(V) + " < " + str(result.GetSolution(rho)) +
      " is in the region of attraction.")
assert(math.fabs(result.GetSolution(rho) - 1) < 1e-5)

Verified that pow(x(0), 2) < 1.00000000717 is in the region of attraction.


# Good luck!

Hopefully that was enough of an initial introduction to help get you started on the orbit transfer problem.

Please ask questions on Piazza and/or come to office hours if you have any more questions.